# Run experiments

### Load packages

In [ ]:
import numpy as np
from qiskit import IBMQ
from qiskit.test.mock import FakeVigo

from analysis.analysis import run_circuits, analyze_results, ExperimentConfiguration, RunConfiguration, PhysicalModel
from analysis.constants import SIMULATOR
from analysis.error_mitigation import CustomErrorMitigation, IgnisErrorMitigation

### Load account

In [ ]:
IBMQ.load_account()
PROVIDER = IBMQ.get_provider(hub='ibm-q-research', group='hu-berlin-1', project='main')
PROVIDER.backends()
BACKEND_HARDWARE = FakeVigo()
BACKEND_SIMULATOR = PROVIDER.get_backend(SIMULATOR)

### Define experiments

In [ ]:
time_vector = np.arange(0, 2, 0.2)
physical_model = PhysicalModel(number_links=4)
print(physical_model)

experiment_configuration = ExperimentConfiguration(zne_extrapolation=False,
                                                   scale_factors=[1.0, 1.5, 2.0], 
                                                   num_replicas=2)
print(experiment_configuration)

run_configuration_simulator = RunConfiguration(time_vector=time_vector,
                                               backend=BACKEND_SIMULATOR)

print(run_configuration_simulator)

### Run experiments

In [ ]:
job_manager_sim, hpc_sim_job_id, circuits_sim = run_circuits(physical_model, experiment_configuration, run_configuration_simulator)

In [ ]:
hpc_res_job = job_manager_sim.retrieve_job_set(job_set_id=hpc_sim_job_id, provider=PROVIDER)

In [ ]:
hpc_res_job.statuses()

#### Retrieve experiments results

In [ ]:
hpc_res_sim = hpc_res_job.results()

The next cell saves the count results from the experiment so they can be recovered later if necessary.

In [ ]:
import pickle

pickle.dump([hpc_res_sim.get_counts(i) for i in range(len(time_vector))], open("my_files", 'wb'))

### Build ignis error filter

In [ ]:
ignis_err_corr = IgnisErrorMitigation(n_qubits=5, shots=1000)
meas_filter = ignis_err_corr.get_meas_fitter(backend=BACKEND_SIMULATOR)


### Analyze results from the experiments

The result is a dataframe with the measurements of all observables based on the experiment configurations

In [ ]:
results_df = analyze_results(physical_model, experiment_configuration, run_configuration_simulator, result_hpc=hpc_res_sim, meas_filter=meas_filter)

In [ ]:
results_df.tail()

### Run experiments using custom error correction.

Only probability density observable is computed

In [ ]:
error_correction = CustomErrorMitigation(n_qubits=5, shots=1000)
error_correction = error_correction.build_probability_matrix(backend=BACKEND_HARDWARE)

In [ ]:
results_df_custom = analyze_results(physical_model, experiment_configuration, run_configuration_simulator, result_hpc=hpc_res_sim, ignis=False, mitigated_counts=error_correction)